In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

### Let us import the required libraries

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sea
import numpy as np
import datetime as dt
import time

##### The data contains columns in json format so we import additional libraries to convert and extract fields inside them

In [ ]:
%%time
import json
from pandas.io.json import json_normalize
import gc
import os

- Loading the data (train and test) and extracting additional information from the json columns simultaneously

In [ ]:
%%time
def load_df(csv_path='../input/train.csv', nrows=300000):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
data=load_df()

In [ ]:
%%time
test=load_df('../input/test.csv')

In [ ]:
data.shape

In [ ]:
test.shape

In [ ]:
data.describe()

In [ ]:
data

- By observing the data above we find the ' socialEngagementType ' column has only one value, let us check the column, if it has only one value delete it.

In [ ]:
data['socialEngagementType'].describe()

###### It has only one value so, we drop the column

In [ ]:
data.drop('socialEngagementType', axis=1, inplace=True)

In [ ]:
data.shape

#### Constant Value Column Filteration
- Most of the columns have nan or a similar type of values (constant), let's check 

In [ ]:
const_cols = [c for c in data.columns if data[c].nunique(dropna=False)==1 ]
const_cols

- Let's try to plot the constant value columns count as percentage of the values inside the column

In [ ]:
data_length = len(data)
def unknown_values(column):
    u_list = ["not available in demo dataset", "(none)", "NaN", "(not provided)", "(not set)", "<NA>", "unknown.unknown"]
    colm_terms = data[column].value_counts()
    for u in u_list:
        if u in colm_terms:
            return colm_terms[u]
    return 0

percent = []
for column in data.columns:
    percent.append(unknown_values(column)/data_length*100.0)

###### Plotting the bar graph

In [ ]:
plt.figure(figsize=(15,15))
plt.barh(data.columns, percent)
plt.xlabel('Missing values in percentage')
plt.title('Percentage missing values for each parameter')
 
plt.show()

- Dropping the constant value columns which have 100% count in the above graph

In [ ]:
data.drop(const_cols, inplace=True, axis=1)

In [ ]:
data.shape

In [ ]:
test.shape

In [ ]:
test.drop(const_cols, inplace=True, axis=1)

In [ ]:
test.shape

- We have required number of Identifiers for each row, so we drop SessionId

In [ ]:
data.drop('sessionId',inplace=True, axis=1)

In [ ]:
data.shape

In [ ]:
test.drop('sessionId',inplace=True, axis=1)

In [ ]:
test.shape

In [ ]:
data.columns

In [ ]:
test.drop('socialEngagementType', axis=1, inplace=True)

In [ ]:
test.columns

###### Check for the columns in train and test dataset
- There should be one less in test dataset, the rest should be same as it affects the training

In [ ]:
for column in data.columns:
    if column not in test.columns:
        print(column)

- There is an extra column in the test dataset 'trafficSource.campaignCode' which is useless for us in the prediction, so we drop it

In [ ]:
data.drop('trafficSource.campaignCode', axis=1, inplace=True)

###### Let's us try to extract the date field into sub fields like month, day, year, weekday
- The date column is not in datetime format, it is given as string with YYYY-MM-DD

In [ ]:
%%time
# This function is to extract date features
def date_format(df):
    df['date'] = df['date'].astype(str) # transforming the date column in string
    df["date"] = df["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:]) # setting a new format to date column yyyy-mm-dd
    df["date"] = pd.to_datetime(df["date"]) # seting the column as pandas datetime
    df["weekday"] = df['date'].dt.weekday #extracting week day
    df["year"] = df['date'].dt.year #extracting the year 
    df["day"] = df['date'].dt.day # extracting day
    df["month"] = df['date'].dt.month #extracting month
    
    return df #returning the df after the transformations

- date_format function is called for both the train and test dataset

In [ ]:
data=date_format(data)

In [ ]:
data.columns

In [ ]:
data.head(2)

In [ ]:
test=date_format(test)

In [ ]:
test.head(2)

In [ ]:
test.describe()

In [ ]:
def normalize(df):
    df['totals.hits'] = df['totals.hits'].astype(float)
    df['totals.newVisits'] = df['totals.newVisits'].astype(float)
    df['totals.pageviews'] = df['totals.pageviews'].astype(float)
    df['totals.transactionRevenue'] = df['totals.transactionRevenue'].astype(float)
    df['totals.transactionRevenue'].fillna(0.0,inplace =True)
    df['totals.transactionRevenue_log'] = (np.log1p(df[df["totals.transactionRevenue"] > 0]["totals.transactionRevenue"]))
    df['totals.transactionRevenue_log'].fillna(0,inplace=True)
    return df

In [ ]:
data=normalize(data)

In [ ]:
data.dtypes

In [ ]:
sea.distplot(data[data['totals.transactionRevenue_log'] > 0.0]['totals.transactionRevenue_log'])

In [ ]:
data.shape

In [ ]:
plt.figure(figsize=(20,15))
sea.heatmap(data.corr(), annot=True)
plt.show()

- The reason Heatmap shows correlation for less fields is that, the columns are not in Integer or float format and correlation is done for these formats.
- Let's convert the totals - subfields into float format.

In [ ]:
def type_convert(df):
    for col in df.columns:
        if df[col].dtype == 'O':
            if col in ['totals.bounces','totals.hits', 'totals.newVisits', 'totals.pageviews']:
                df[col] = df[col].astype('float')
            else:
                df[col] = df[col].astype('str')
    return df

In [ ]:
# data.dtypes

In [ ]:
# %%time
# data=type_convert(data)

In [ ]:
# data.dtypes

In [ ]:
plt.figure(figsize=(20,15))
sea.heatmap(data.corr(), annot=True)
plt.show()

- Let us add another feature revenue_status to indicate presence of revenue for each row.

In [ ]:
data['revenue_status']=data['totals.transactionRevenue'].apply(lambda x: 0 if x==0 else 1)

In [ ]:
data.columns

##### Let us see what percent of the transactions generate revenue.

In [ ]:
# # Visualization for Visits by date
# tmp = data['date'].value_counts().to_frame().reset_index().sort_values('index')
# tmp = tmp.rename(columns = {"index" : "dateX", "date" : "visits"})

# tr = go.Scatter(mode="lines", x = tmp["dateX"].astype(str), y = tmp["visits"])
# layout = go.Layout(title="Visits by Date", height=400)
# fig = go.Figure(data = [tr], layout = layout)
# iplot(fig)
# # Visualization for Visits by monthly revenue
# tmp = data.groupby("date").agg({"totals_transactionRevenue" : "mean"}).reset_index()
# tmp = tmp.rename(columns = {"date" : "dateX", "totals_transactionRevenue" : "mean_revenue"})
# tr = go.Scatter(mode="lines", x = tmp["dateX"].astype(str), y = tmp["mean_revenue"])
# layout = go.Layout(title="Monthly Revenue by Date", height=400)
# fig = go.Figure(data = [tr], layout = layout)
# iplot(fig)

In [ ]:
pie_labels=['Revenue Generated -'+str(data['revenue_status'][data.revenue_status==1].count()),'No Revenue Generated-'+
            str(data['revenue_status'][data.revenue_status==0].count())]

pie_data=[data['revenue_status'][data.revenue_status==1].count()/data['revenue_status'].count(),
           data['revenue_status'][data.revenue_status==0].count()/data['revenue_status'].count()]

fig1, ax1 = plt.subplots(figsize=(6,6))
explode=(.30,.15)

ax1.pie(pie_data,labels=pie_labels,explode=explode,autopct='%.2f%%',startangle=45,shadow=True)
ax1.axis('equal')

plt.title('Percentage of Transactions Generating Revenue and Not Generating Revenue',color='blue',fontsize=12)
plt.show()


- From the above pie chart we can clearly understnd that the recenue is generated by only 11515 records!

##### Let us plot Number Of Transations Generating Revenue by Browser

In [ ]:
df_browser=data[['device.browser','totals.transactionRevenue','revenue_status']].groupby(data['device.browser']).aggregate({'totals.transactionRevenue':['mean'],
                                                                                                              'revenue_status':['count']}).reset_index()
df_browser.columns=['device.browser','totals.transactionRevenue_mean','revenue_status_count']
df=df_browser.sort_values(by='totals.transactionRevenue_mean',ascending=False)[df_browser['totals.transactionRevenue_mean']>0]
formatter = {'totals.transactionRevenue_mean':'{:4.2f}'}
display(df.style.format(formatter))

In [ ]:
plt.subplots(figsize=(22,6))
plt.subplot(1,2,1)
plt.title('REVENUE MEAN BY BROWSER',fontsize=18)
plt.xlabel('Browser Type',fontsize=18)
plt.ylabel('Mean Revenue',fontsize=18)
plt.bar(range(len(df)),df['totals.transactionRevenue_mean'],color='c')
plt.xticks(range(len(df)),df['device.browser'],rotation=90,fontsize=12)
plt.yticks(fontsize=12)


plt.subplot(1,2,2)
plt.title('NUMBER OF TRANSACTIONS WITH REVENUE BY BROWSER',fontsize=18)
plt.xlabel('Browser Type',fontsize=18)
plt.ylabel('Number of Transactions with Revenue',fontsize=18)
plt.bar(range(len(df)),df.revenue_status_count,color='orange')
plt.xticks(range(len(df)),df['device.browser'],rotation=90,fontsize=12)
plt.yticks(fontsize=12)
plt.show()

###### By analyzing the above plots we can infer that,
- Revenue is generated by only 9 browsers.
- The Maximum mean revenue is generated using firefox.
- The Maximum transactions with revenue are done using chrome.
- These indicates that only few browsers are more important in revenue generation.

##### Let us plot Number Of Transations Generating Revenue by Operating System.

In [ ]:
df_OS=data[['device.operatingSystem','totals.transactionRevenue','revenue_status']].groupby(data['device.operatingSystem']).aggregate({'totals.transactionRevenue':['mean'],
                                                                                                              'revenue_status':['count']}).reset_index()
df_OS.columns=['device.operatingSystem','totals.transactionRevenue_mean','revenue_status_count']
df=df_OS.sort_values(by='totals.transactionRevenue_mean',ascending=False)[df_OS['totals.transactionRevenue_mean']>0]
display(df.style.format(formatter))

In [ ]:
plt.subplots(figsize=(22,6))
plt.subplot(1,2,1)
plt.title('REVENUE MEAN BY OPERATING SYSTEM',fontsize=18)
plt.xlabel('Operating Systems',fontsize=18)
plt.ylabel('Mean Revenue',fontsize=18)
plt.bar(range(len(df)),df['totals.transactionRevenue_mean'],color='c')
plt.xticks(range(len(df)),df['device.operatingSystem'],rotation=90,fontsize=12)
plt.yticks(fontsize=12)


plt.subplot(1,2,2)
plt.title('NUMBER OF TRANSACTIONS WITH REVENUE BY OS',fontsize=18)
plt.xlabel('Operating Systems',fontsize=18)
plt.ylabel('Count of Transactions with Revenue',fontsize=18)
plt.bar(range(len(df)),df['revenue_status_count'],color='orange')
plt.yticks(fontsize=12)
plt.xticks(range(len(df)),df['device.operatingSystem'],rotation=90,fontsize=12)
plt.show()

###### By analyzing the above plots we can infer that,
- Revenue is generated by only 7 browsers.
- The Maximum mean revenue is generated using Chrome OS.
- The Maximum transactions with revenue are done using Windows OS.
- These indicates that only few OS are more important in revenue generation.

##### Let us plot Number Of Transations Generating Revenue by Operating System.

In [ ]:
df_M=data[['device.isMobile','totals.transactionRevenue','revenue_status']].groupby(data['device.isMobile']).aggregate({'totals.transactionRevenue':['mean'],
                                                                                                              'revenue_status':['count']}).reset_index()
df_M.columns=['device.isMobile','totals.transactionRevenue_mean','revenue_status_count']
df=df_M.sort_values(by='totals.transactionRevenue_mean',ascending=False)[df_M['totals.transactionRevenue_mean']>0]
display(df.style.format(formatter))

In [ ]:
plt.subplots(figsize=(22,6))
plt.subplot(1,2,1)
plt.title('REVENUE MEAN BY MOBILE',fontsize=18)
plt.xlabel('Mobile',fontsize=18)
plt.ylabel('Mean Revenue',color='b',fontsize=18)
plt.bar(range(len(df)),df['totals.transactionRevenue_mean'],color='c')
plt.xticks(range(len(df)),df['device.isMobile'],rotation=90,fontsize=15)
plt.yticks(fontsize=12)


plt.subplot(1,2,2)
plt.title('NUMBER OF TRANSACTIONS WITH REVENUE BY MOBILE',fontsize=18)
plt.xlabel('Mobile',fontsize=18)
plt.ylabel('Count of Transactions with Revenue',fontsize=18)
plt.bar(range(len(df)),df['revenue_status_count'],color='orange')
plt.yticks(fontsize=12)
plt.xticks(range(len(df)),df['device.isMobile'],rotation=90,fontsize=15)
plt.show()

###### By analyzing the above plots we can infer that,
- More revenue is generated from non mobile devices.
- More number of transac

In [ ]:
data.shape

##### Let us plot Number Of Transations Generating Revenue by Country.

In [ ]:
df_country=data[['geoNetwork.country','totals.transactionRevenue','revenue_status']].groupby(data['geoNetwork.country']).aggregate({'totals.transactionRevenue':['mean'],
                                                                                                              'revenue_status':['count']}).reset_index()
df_country.columns=['geoNetwork.country','totals.transactionRevenue_mean','revenue_status_count']
df=df_country.sort_values(by='totals.transactionRevenue_mean',ascending=False)[:20]
display('The following is Sorted by Mean Transaction Revenue')
display(df.style.format(formatter))


plt.subplots(figsize=(22,6))
plt.subplot(1,2,1)
plt.title('REVENUE MEAN BY COUNTRY',fontsize=18)
plt.xlabel('Country',fontsize=18)
plt.ylabel('Mean Revenue',fontsize=18)
plt.bar(range(len(df)),df['totals.transactionRevenue_mean'],color='c')
plt.xticks(range(len(df)),df['geoNetwork.country'],rotation=90,fontsize=15)
plt.yticks(fontsize=12)


plt.subplot(1,2,2)
plt.title('NUMBER OF TRANSACTIONS WITH REVENUE BY COUNTRY',fontsize=18)
plt.xlabel('Country',fontsize=18)
plt.ylabel('Count of Transactions with Revenue',fontsize=18)
plt.bar(range(len(df)),df['revenue_status_count'],color='orange')
plt.yticks(fontsize=12)
plt.xticks(range(len(df)),df['geoNetwork.country'],rotation=90,fontsize=15)
plt.show()

- The above plot is plotted using values sorted by mean transaction revenue.
- It is clear that Anguilla has the highest mean revenue.


- Now let us plot using the values sorted by Count of transactions with revenue.

In [ ]:
df1=df_country.sort_values(by='revenue_status_count',ascending=False)[:20]
display('The following is sorted by Count of Transactions with Revenue')
display(df1.style.format(formatter))

plt.subplots(figsize=(22,6))
plt.subplot(1,2,1)
plt.title('REVENUE MEAN BY COUNTRY',fontsize=18)
plt.xlabel('Country',fontsize=18)
plt.ylabel('Mean Revenue',fontsize=18)
plt.bar(range(len(df1)),df1['totals.transactionRevenue_mean'],color='c')
plt.xticks(range(len(df1)),df1['geoNetwork.country'],rotation=90,fontsize=15)
plt.yticks(fontsize=12)


plt.subplot(1,2,2)
plt.title('NUMBER OF TRANSACTIONS WITH REVENUE BY COUNTRY',fontsize=18)
plt.xlabel('Country',fontsize=18)
plt.ylabel('Count of Transactions with Revenue',fontsize=18)
plt.bar(range(len(df1)),df1['revenue_status_count'],color='orange')
plt.yticks(fontsize=12)
plt.xticks(range(len(df1)),df1['geoNetwork.country'],rotation=90,fontsize=15)
plt.show()

- United states has the highest number of transactions which generate revenue.

##### Let us plot Number Of Transations Generating Revenue by City.

In [ ]:
df_city=data[['geoNetwork.city','totals.transactionRevenue','revenue_status']].groupby(data['geoNetwork.city']).aggregate({'totals.transactionRevenue':['mean'],
                                                                                                              'revenue_status':['count']}).reset_index()
df_city.columns=['geoNetwork.city','totals.transactionRevenue_mean','revenue_status_count']
df=df_city.sort_values(by='totals.transactionRevenue_mean',ascending=False)[:20]
display('The following are sorted by mean transaction revenue')
display(df)

plt.subplots(figsize=(22,6))
plt.subplot(1,2,1)
plt.title('REVENUE MEAN BY CITY',fontsize=18)
plt.xlabel('City',fontsize=18)
plt.ylabel('Mean Revenue',fontsize=18)
plt.bar(range(len(df)),df['totals.transactionRevenue_mean'],color='c')
plt.xticks(range(len(df)),df['geoNetwork.city'],rotation=90,fontsize=15)
plt.yticks(fontsize=12)


plt.subplot(1,2,2)
plt.title('NUMBER OF TRANSACTIONS WITH REVENUE BY CITY',fontsize=18)
plt.xlabel('City',fontsize=18)
plt.ylabel('Count of Transactions with Revenue',fontsize=18)
plt.bar(range(len(df)),df['revenue_status_count'],color='orange')
plt.yticks(fontsize=12)
plt.xticks(range(len(df)),df['geoNetwork.city'],rotation=90,fontsize=15)
plt.show()

In [ ]:
df1=df_city.sort_values(by='revenue_status_count',ascending=False)[:20]
display('These are sorted by count of Transactions with Revenue')
display(df1)

plt.subplots(figsize=(22,6))
plt.subplot(1,2,1)
plt.title('REVENUE MEAN BY CITY',fontsize=18)
plt.xlabel('City',fontsize=18)
plt.ylabel('Mean Revenue',fontsize=18)
plt.bar(range(len(df1)),df1['totals.transactionRevenue_mean'],color='c')
plt.xticks(range(len(df1)),df1['geoNetwork.city'],rotation=90,fontsize=15)
plt.yticks(fontsize=12)


plt.subplot(1,2,2)
plt.title('NUMBER OF TRANSACTIONS WITH REVENUE BY CITY',fontsize=18)
plt.xlabel('City',fontsize=18)
plt.ylabel('Count of Transactions with Revenue',fontsize=18)
plt.bar(range(len(df1)),df1['revenue_status_count'],color='orange')
plt.yticks(fontsize=12)
plt.xticks(range(len(df1)),df1['geoNetwork.city'],rotation=90,fontsize=15)
plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(8,5))
# ax = sea.regplot(x=data['visitNumber'], y=data['totals.transactionRevenue'], scatter_kws={"s": 10})
# plt.show()

In [ ]:
test.dtypes

In [ ]:
train1 = data
test1 = test

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

In [ ]:
# data['totals.transactionRevenue'].fillna(0, inplace=True)

col_list = ['channelGrouping', 'device.browser', 'device.deviceCategory',
       'device.isMobile', 'device.operatingSystem', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
       'geoNetwork.networkDomain',
       'geoNetwork.subContinent',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
#        'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
#        'trafficSource.isTrueDirect',
        'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source']

In [ ]:
data['totals.transactionRevenue'].describe()

In [ ]:
%%time

for col in col_list:
    label = preprocessing.LabelEncoder()
    label.fit(list(data[col]) + list(test[col]))
    data[col] = label.transform(list(data[col]))
    test[col] = label.transform(list(test[col]))
    print(col + "\t------\tDone\t------")

In [ ]:
plt.figure(figsize=(30,25))
sea.heatmap(data.corr(), annot=True)
plt.show()

In [ ]:
data.columns

In [ ]:
train_var = ['visitNumber', 'device.deviceCategory', 'device.browser', 'device.isMobile', 'device.operatingSystem', 'geoNetwork.continent',
        'geoNetwork.country',  'totals.hits',
        'totals.pageviews']

In [ ]:
data['totals.transactionRevenue'] = np.log1p(data['totals.transactionRevenue'])
df_x = data[train_var]
# df_x = df_x.drop(columns = ['totals.transactionRevenue', 'date'])
df_y = data['totals.transactionRevenue']

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 3)

In [ ]:
import xgboost as xg

In [ ]:
# path = '/Macintosh HD⁩/Users⁩/peela⁩/⁨Documents/Dsf⁩/Assignment2⁩/all⁩/sub.csv'

In [ ]:
XGB = xg.XGBRegressor(n_estimators=100, learning_rate=0.09, subsample=0.75, max_depth=24, n_jobs = 8)

In [ ]:
%%time

XGB.fit(x_train,y_train)

In [ ]:
%%time

ans = XGB.predict(x_test)

In [ ]:
mse = np.mean((ans-y_test)**2)
print('mean square error')
print(mse)
# root mean square error
rmse = np.sqrt(mse)
print('root mean square error')
print(rmse)

In [ ]:
ans.min()

In [ ]:
XGB.fit(data[train_var],data['totals.transactionRevenue'])

In [ ]:
data['totals.transactionRevenue'].describe()

In [ ]:
test.dtypes

In [ ]:
test=type_convert(test)

In [ ]:
p = XGB.predict(test[train_var])

In [ ]:
p.mean()

In [ ]:
submission = pd.DataFrame()
submission['fullVisitorId']=test['fullVisitorId']
# p[p<0]=0
submission['PredictedLogRevenue'] = np.expm1(p)
submission = submission.groupby('fullVisitorId')['PredictedLogRevenue'].sum().reset_index()
submission['PredictedLogRevenue'] = np.log1p(submission['PredictedLogRevenue'])

submission.to_csv('submission3_mod_1.csv', index=False)
submission.tail(20)

In [ ]:
submission.columns